<img src="images/logo_city.png" align="right" width="20%">

# Data Generation Module Development

## 1. API

Import necessary libraries:

In [ ]:
import timeslice                    
import timeslice.source as source   
import timeslice.rule  as rule      
import timeslice.worker as worker
import timeslice.viz as viz # to be done
import torch

Define tensor generator:

In [ ]:
# set time split rule for dataset generation
time_rule = rule.TimeSlice(stp='2017-01-01 00:00:00', etp='2018-01-01 00:00:00', freq='15min')

# connect to database source
taxi = source.DatabaseSource('cleaned_small_yellow_2017_full', time_rule)

# initialize worker
tensor_gen = worker.Worker(source=taxi, destin='data_test/concurrent_test', rule=time_rule, viz=True)

Generate tensors:

In [ ]:
tensor_gen.parallel_gen()